## Lana labs Data Science Assignment - Jiacheng Yao (jc07.yao@gmail.com)

#### Q3.4: (Super Bonus): Add something fun to your analysis that has something to do with penguins. For example, you may fast forward the starting date of your events to last week and use the standard Twitter search API to get the number of tweets about penguins for each day. This you could add as an extra column to each event so we exactly know how much people talked about penguins while our process was running :-)

In [1]:
import pandas as pd
import numpy as np
import datetime

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style('whitegrid')

##### Obtain event data and do preprocessing

In [2]:
# Read the input data
df_raw = pd.read_csv("Example MIX TYPES - Lana Labs.txt", sep="\t")

In [3]:
# Fix point provided by the client
time_base = datetime.datetime(2019, 7, 8, 10, 0, 0)

# Add a new column called TimeStamp that stores datetime format timestamp data
df_raw['TimeStamp'] = df_raw.apply(lambda row: time_base + datetime.timedelta(0, row[u'Time']), axis=1)

# sort the original dataset by ['ActivityName', 'eLetter_ID', 'Time'] 
# 1so that all entries with the same ['ActivityName', 'eLetter_ID'] will be adjacent to each other 
df_raw.sort_values(by=['ActivityName', 'eLetter_ID', 'TimeStamp'], inplace=True)

# Transform the original data into the format that we want in the end
tmp_event_ID = 0
col_names =  ['ActivityName', 'eLetter_ID', 'Event_ID', 'eLetter_Type', 'Start', 'Complete']
df_transformed = pd.DataFrame(columns = col_names)
tmp_activity = None
tmp_eLetter = None

df_raw.iloc[[0]]
for index, row in df_raw.iterrows():
    if index == 0:
        tmp_activity = row['ActivityName']
        tmp_eLetter = row['eLetter_ID']
        
        df_transformed = df_transformed.append({
            'ActivityName' : row['ActivityName'], 
            'eLetter_ID' : row['eLetter_ID'],
            'Event_ID': tmp_event_ID,
            'eLetter_Type': row['eLetter_Type'],
            'Start': row['TimeStamp']
        } , ignore_index=True)
        tmp_event_ID = tmp_event_ID + 1
    else:
        if (row['ActivityName']==tmp_activity and row['eLetter_ID']==tmp_eLetter):
            df_transformed.loc[df_transformed.index[-1], 'Complete']= row['TimeStamp']
        else:
            tmp_activity = row['ActivityName']
            tmp_eLetter = row['eLetter_ID']
        
            tmp_event_ID = tmp_event_ID + 1
            df_transformed = df_transformed.append({
                'ActivityName' : row['ActivityName'], 
                'eLetter_ID' : row['eLetter_ID'],
                'Event_ID': tmp_event_ID,
                'eLetter_Type': row['eLetter_Type'],
                'Start': row['TimeStamp']
            } , ignore_index=True)
            
df_transformed_filtered = df_transformed[pd.notnull(df_transformed['Complete'])]

In [4]:
df_transformed_filtered.head()

,ActivityName,eLetter_ID,Event_ID,eLetter_Type,Start,Complete
0,Create Initial Draft,1,1,Type_1,2019-07-09 10:04:23.936862,2019-07-09 15:33:02.944338
1,Create Initial Draft,2,2,Type_2,2019-07-09 10:35:00.000000,2019-07-09 15:41:30.731234
2,Create Initial Draft,4,3,Type_4,2019-07-09 11:15:00.000000,2019-07-10 13:45:31.909560
3,Create Initial Draft,5,4,Type_1,2019-07-09 10:25:00.000000,2019-07-09 15:05:20.828435
4,Create Initial Draft,13,5,Type_1,2019-07-10 10:25:00.000000,2019-07-10 13:50:16.543405


##### Extract Twitter Data from the last week and aggregate: sum up all the favorites and group by date

In [5]:
# Import the Twython class
from twython import Twython  
import json

# Load credentials from json file
with open("twitter_credentials.json", "r") as file:  
    creds = json.load(file)

# Instantiate an object
python_tweets = Twython(creds['CONSUMER_KEY'], creds['CONSUMER_SECRET'])


# Create our query
query = {'q': 'penguins',  
        'result_type': 'popular',
        'count': 100,
        'lang': 'en',
        'until': '2019-07-15'
        }


# Search tweets
dict_ = {'user': [], 'date': [], 'text': [], 'favorite_count': []}  
for status in python_tweets.search(**query)['statuses']:  
    dict_['user'].append(status['user']['screen_name'])
    dict_['date'].append(status['created_at'])
    dict_['text'].append(status['text'])
    dict_['favorite_count'].append(status['favorite_count'])

# Structure data in a pandas DataFrame for easier manipulation
df_twitter = pd.DataFrame(dict_)  
df_twitter.sort_values(by='favorite_count', inplace=True, ascending=False)  

import pytz
df_twitter['date_formatted'] = df_twitter.apply(lambda row: datetime.datetime.strptime(row['date'],'%a %b %d %H:%M:%S +0000 %Y').replace(tzinfo=pytz.UTC), axis=1)
df_twitter['Datum'] = df_twitter.apply(lambda row: row['date_formatted'].strftime('%Y-%m-%d'), axis=1)
df_twitter_aggregate = df_twitter.groupby('Datum').agg({'favorite_count': ['sum']}).reset_index()
df_twitter_aggregate.columns = ['Datum', 'favorite_count']

In [6]:
df_twitter_aggregate

,Datum,favorite_count
0,2019-07-08,80093
1,2019-07-09,2654
2,2019-07-10,25733
3,2019-07-12,968
4,2019-07-13,5245
5,2019-07-14,89


In [7]:
df_lanalabs = df_transformed_filtered.copy()
df_lanalabs['Datum'] = df_lanalabs.apply(lambda row: row['Start'].strftime('%Y-%m-%d'), axis=1)

In [8]:
df_final = pd.merge(df_lanalabs, df_twitter_aggregate,  how='left', left_on=['Datum'], right_on = ['Datum'])

In [9]:
df_final.head(15)

,ActivityName,eLetter_ID,Event_ID,eLetter_Type,Start,Complete,Datum,favorite_count
0,Create Initial Draft,1,1,Type_1,2019-07-09 10:04:23.936862,2019-07-09 15:33:02.944338,2019-07-09,2654.0
1,Create Initial Draft,2,2,Type_2,2019-07-09 10:35:00.000000,2019-07-09 15:41:30.731234,2019-07-09,2654.0
2,Create Initial Draft,4,3,Type_4,2019-07-09 11:15:00.000000,2019-07-10 13:45:31.909560,2019-07-09,2654.0
3,Create Initial Draft,5,4,Type_1,2019-07-09 10:25:00.000000,2019-07-09 15:05:20.828435,2019-07-09,2654.0
4,Create Initial Draft,13,5,Type_1,2019-07-10 10:25:00.000000,2019-07-10 13:50:16.543405,2019-07-10,25733.0
5,Create Initial Draft,14,6,Type_2,2019-07-10 10:35:00.000000,2019-07-10 16:09:04.398857,2019-07-10,25733.0
6,Create Initial Draft,16,7,Type_4,2019-07-10 11:20:34.634009,2019-07-11 15:56:14.879615,2019-07-10,25733.0
7,Create Initial Draft,17,8,Type_1,2019-07-10 17:07:23.657716,2019-07-11 11:58:55.425826,2019-07-10,25733.0
8,Create Initial Draft,25,9,Type_1,2019-07-11 17:08:04.332226,2019-07-12 12:13:21.465206,2019-07-11,NaN
9,Create Initial Draft,26,10,Type_2,2019-07-12 10:35:00.000000,2019-07-12 13:56:26.205356,2019-07-12,968.0


In [10]:
# Write the final results to file
df_final.to_csv("lanalabs_final_output.csv", sep='\t', encoding='utf-8')